As always, start by import `datajoint` library.

In [ ]:
import datajoint as dj

Create a schema to define tables in. Be sure to replace `'YOUR_USERNAME'` with your Database username (e.g. if your user name is `john`, make it `john_calcium`)

In [ ]:
schema = dj.schema('dln_YOUR_USERNAME_calcium')

We now create a table for Mouse

In [ ]:
@schema
class Mouse(dj.Manual):
    definition = """
    # Experimental animals
    mouse_id             : int                          # Unique animal ID
    ---
    dob=null             : date                         # date of birth
    sex="unknown"        : enum('M','F','unknown')      # sex
    mouse_notes=""       : varchar(4096)                # other comments and distinguishing features
    """

and insert a few mice into the newly defined table. You can either insert 1 at a time...

In [ ]:
Mouse.insert1(
{'dob': '2016-01-05',
 'mouse_id': 8602,
 'mouse_notes': '',
 'sex': 'M'})
Mouse.insert1({'dob': '2016-01-06',
 'mouse_id': 8603,
 'mouse_notes': '',
 'sex': 'M'})

or many togethr in a batch.

In [ ]:
Mouse.insert([
    {'dob': "2016-01-04", 'mouse_id': 8623, 'sex': 'M'},
    {'dob': "2016-01-22", 'mouse_id': 8804, 'sex': 'F'}])

We now define a table for scan sessions.

In [ ]:
@schema
class Session(dj.Manual):
    definition = """
    -> Mouse
    session              : smallint                     # session number
    ---
    session_date         : date                         # date
    person               : varchar(100)                 # researcher name
    scan_path            : varchar(255)                 # file path for TIFF stacks
    """

We can look at tables and their relationships by displaying the entitiy relationship diagram (ERD).

In [ ]:
dj.Di(schema)

Let's try inserting a few sessions manually.

In [ ]:
Session.insert1(
    {'mouse_id': 8623, 
     'session': 1, 
     'session_date': "2016-02-17",
     'person': 'Jake', 
     'scan_path': '/scratch01/Two-Photon/160217'
    })

Session.insert1(
    {   'mouse_id': 8804,
        'person': 'Manolis',
        'scan_path': '/scratch03/Two-Photon/Jake/160302',
        'session': 1,
        'session_date': "2016-03-02"})

In [ ]:
Mouse()

We can select for all mouse for which one or more session exist.

In [ ]:
Mouse & Session

In [ ]:
(Mouse - {'mouse_id': 8623}).delete()

In [ ]:
Mouse * Session

In [ ]:
males = Mouse * Session & 'sex="M"'

In [ ]:
males.proj('person', 'scan_path')

In [ ]:
a = males.fetch()

In [ ]:
Mouse.drop()

In [ ]:
@schema
class EphysRecording(dj.Manual):
    definition = """
    # Intracellular 
    -> Session
    rec : int   # scan number within session
    ---
    duration  :    float  # (s)
    """

In [ ]:
EphysRecording()

In [ ]:
@schema
class BrainArea(dj.Lookup):
    definition = """
    # brain areas for recording
    brain_area :  varchar(20)   
    """
    contents = [['V1'], ['S1'], ['LM'], ['A1']]

In [ ]:
BrainArea()